In [ ]:
import manganite
%load_ext manganite

# Airfoil Analysis
# with [AeroSandbox](https://github.com/peterdsharpe/AeroSandbox) and [Manganite](https://github.com/LCL-CAVE/manganite)

## Description

The Airfoil Design Demonstration Dashboard is a web-based showcase that provides a glimpse into the capabilities of airfoil analysis within a browser environment. It is built upon the open-source repository [AeroSandbox](https://github.com/peterdsharpe/Automotive-Airfoil-Design/) and serves as a demonstration of what's possible in the realm of virtual airfoil exploration.

### Key Features

1. **Geometry Visualization:** Explore an interactive airfoil geometry, pre-defined for demonstration purposes. Observe how changes in shape and size can impact aerodynamic behavior.

2. **Simplified Angle of Attack:** Adjust the angle of attack within a limited range to see the immediate effects on lift and drag forces. 

3. **Kulfan Coordinates:** This demonstration employs Kulfan coordinates for the representation of airfoil shapes, providing insight into how airfoil data can be structured and analyzed.

4. **Basic Constant Display:** View simplified constant values such as lift coefficient (CL), drag coefficient (CD), and moment coefficient (CM).

5. **Visualization:** Visualize how changes in airfoil geometry and angle of attack influence aerodynamic characteristics.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import aerosandbox as asb  #requires fixed version aerosandbox==3.0.0
import aerosandbox.numpy as np
import copy
import plotly.figure_factory as ff
import pandas as pd

In [ ]:
%%mnn widget --type slider -20:20:0.1 --tab "Operating Conditions" --header "Angle of Attack" --var alpha
alpha = 8.7

In [ ]:
streamline_density = 1
height = 0
ground_effect = False

In [ ]:
%%mnn widget --type slider -0.1:0.7:0.01 --tab "Shape Parameters" --header "Upper surface 1" --var upper_1 --position 0 0 2
upper_1 = 0.25

In [ ]:
%%mnn widget --type slider -0.1:0.7:0.01 --tab "Shape Parameters" --header "Upper surface 2" --var upper_2 --position 0 2 2
upper_2 = 0.47

In [ ]:
%%mnn widget --type slider -0.1:0.7:0.01 --tab "Shape Parameters" --header "Upper surface 3" --var upper_3 --position 0 4 2
upper_3 = 0.024

In [ ]:
%%mnn widget --type slider -0.5:0.3:0.01 --tab "Shape Parameters" --header "Lower surface 1" --var lower_1 --position 1 0 2
lower_1 = -0.11

In [ ]:
%%mnn widget --type slider 0:0.7:0.01 --tab "Shape Parameters" --header "Lower surface 2" --var lower_2 --position 1 2 2
lower_2 = 0.06

In [ ]:
%%mnn widget --type slider 0:0.7:0.01 --tab "Shape Parameters" --header "Lower surface 3" --var lower_3 --position 1 4 2
lower_3 = -0.06

In [ ]:
def display_graph(n_clicks, alpha, height, streamline_density, kulfan_upper, kulfan_lower, analyze_button_pressed = False):

    ### Start constructing the figure
    airfoil = asb.Airfoil(
        coordinates=asb.get_kulfan_coordinates(
            lower_weights=np.array(lower_values),
            upper_weights=np.array(upper_values),
            TE_thickness=0,
            enforce_continuous_LE_radius=False,
            n_points_per_side=200
        )
    )

    # ### Do coordinates output
    # coordinates_output = "\n".join(
    #     ["```"] +
    #     ["AeroSandbox Airfoil"] +
    #     ["\t%f\t%f" % tuple(coordinate) for coordinate in airfoil.coordinates] +
    #     ["```"]
    # )

    ### Continue doing the airfoil things
    airfoil = airfoil.rotate(angle=-np.radians(alpha))
    airfoil = airfoil.translate(
        0,
        height + 0.5 * np.sind(alpha)
    )
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=airfoil.x(),
            y=airfoil.y(),
            mode="lines",
            name="Airfoil",
            fill="toself",
            line=dict(
                color="blue"
            )
        )
    )

    ### Default text output
    text_output = 'Click "Analyze" to compute aerodynamics!'
    output = text_output

    xrng = (-0.5, 1.5)
    yrng = (-0.6, 0.6) if not ground_effect else (0, 1.2)

    if analyze_button_pressed:

        analysis = asb.AirfoilInviscid(
            airfoil=airfoil.repanel(50),
            op_point=asb.OperatingPoint(
                velocity=1,
                alpha=0,
            ),
            ground_effect=ground_effect
        )

        x = np.linspace(*xrng, 100)
        y = np.linspace(*yrng, 100)
        X, Y = np.meshgrid(x, y)
        u, v = analysis.calculate_velocity(
            x_field=X.flatten(),
            y_field=Y.flatten()
        )
        U = u.reshape(X.shape)
        V = v.reshape(Y.shape)

        streamline_fig = ff.create_streamline(
            x, y, U, V,
            arrow_scale=1e-16,
            density=streamline_density,
            line=dict(
                color="#ff82a3"
            ),
            name="Streamlines"
        )

        fig = go.Figure(
            data=streamline_fig.data + fig.data
        )

        output = pd.DataFrame(
            {
                "Engineering Quantity": [
                    "C_L"
                ],
                "Value"               : [
                    f"{analysis.Cl:.3f}"
                ]
            }
        )

    fig.update_layout(
        xaxis_title="x/c",
        yaxis_title="y/c",
        showlegend=False,
        yaxis=dict(scaleanchor="x", scaleratio=1),
        margin={'t': 0},
        title=None,
    )

    fig.update_xaxes(range=xrng)
    fig.update_yaxes(range=yrng)

    return fig, output

In [ ]:
%%mnn widget --type plot --var aero_fig --tab "Shape Parameters" --header "Airfoil Cross section" --position 2 0 6
upper_values = [upper_1, upper_2,upper_3]
lower_values = [lower_1, lower_2, lower_3]
aero_fig, text_output = display_graph(1,alpha=alpha,height=0,streamline_density=streamline_density, kulfan_upper=upper_values, kulfan_lower=lower_values)


In [ ]:
%%mnn execute --on button "Analyze" --returns aero_performance

aero_fig_lines, aero_performance = display_graph(1,alpha=alpha,height=0,streamline_density=streamline_density, kulfan_upper=upper_values, kulfan_lower=lower_values, analyze_button_pressed = True)


In [ ]:
%%mnn widget --type plot --var aero_fig_lines --tab "Aerodynamic Performance" --header "Streamlines" --position 0 0 6

simu_ready = aero_performance
aero_fig_lines


In [ ]:
%%mnn widget --type table --tab "Aerodynamic Performance" --position 1 2 4 --header "Metrics" --var performance_values

performance_values = aero_performance